In [2]:
import pymysql #데이터베이스 연결
import numpy as np #수치 계산
from scipy.spatial.distance import euclidean, cosine #거리 계산

In [3]:
# user_record와 album 테이블에서 필요한 데이터를 가져오는 함수
def fetch_records_and_albums(connection):
    with connection.cursor() as cursor: #데이터베이스 커서가 생성되고, 블록이 끝나면 자동으로 닫힘
        cursor.execute("SELECT user_id, album_id FROM user_record") # user_record 테이블에서 user_id와 album_id를 가져오기 위한 SQL 쿼리 실행
        records = cursor.fetchall()  # 쿼리 결과를 records 변수에 저장
        
        cursor.execute("""
            SELECT id, music_acousticness, music_danceability, music_energy, 
                   music_liveness, music_loudness, music_speechiness, 
                   music_tempo, music_valence 
            FROM album
        """) # album 테이블에서 id와 여러 특성 값을 가져오기 위한 SQL 쿼리 실행
        albums = cursor.fetchall()  # 쿼리 결과를 albums 변수에 저장
    return records, albums  # 가져온 데이터를 반환

In [ ]:
def main():
    connection = pymysql.connect(
        host='mitidb.cvm64ss6y2xv.ap-northeast-2.rds.amazonaws.com',       
        user='minseo',  
        password='Alstj!!809', 
        database='mitiDB'
    ) # MySQL 연결

if __name__ == "__main__":
    main()